In [1]:
import pandas as pd

basics = pd.read_csv('title.basics.tsv', sep='\t', low_memory=False, na_values=["\\N","nan"])
basics = basics[basics["titleType"] == "movie"]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0.0,1894.0,NaN,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0.0,1897.0,NaN,100,"Documentary,News,Sport"
498,tt0000502,movie,Bohemios,Bohemios,0.0,1905.0,NaN,100,NaN
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906.0,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0.0,1907.0,NaN,90,Drama
...,...,...,...,...,...,...,...,...,...
9629481,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0.0,2015.0,NaN,57,Documentary
9629508,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0.0,2007.0,NaN,100,Documentary
9629520,tt9916706,movie,Dankyavar Danka,Dankyavar Danka,0.0,2013.0,NaN,NaN,Comedy
9629530,tt9916730,movie,6 Gunn,6 Gunn,0.0,2017.0,NaN,116,NaN


In [2]:
ratings = pd.read_csv('title.ratings.tsv', sep='\t', low_memory=False, na_values=["\\N","nan"])
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1952
1,tt0000002,5.8,264
2,tt0000003,6.5,1787
3,tt0000004,5.6,179
4,tt0000005,6.2,2589
...,...,...,...
1281760,tt9916730,8.3,10
1281761,tt9916766,7.0,21
1281762,tt9916778,7.2,36
1281763,tt9916840,8.8,6


In [3]:
cast = pd.read_csv('title.principals.tsv', sep='\t', low_memory=False, na_values=["\\N","nan"])
names = pd.read_csv('name.basics.tsv', sep='\t', low_memory=False, na_values=["\\N","nan"])

In [4]:
df = pd.merge(basics, ratings, on='tconst')
df = df.sort_values(by='numVotes', ascending=False)

top = df.loc[df['primaryTitle'] == df['originalTitle']].head(2000).merge(
    cast.loc[cast['category'].isin(['actor','actress'])], on='tconst'
).merge(names, on='nconst').groupby(['tconst']).agg({
    'primaryName': '|'.join
}).merge(basics, on='tconst')

top

,tconst,primaryName,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0012349,Charles Chaplin|Henry Bergman|B.F. Blinn|Edna ...,movie,The Kid,The Kid,0.0,1921.0,NaN,68,"Comedy,Drama,Family"
1,tt0015864,Charles Chaplin|Henry Bergman|Frank Aderias|Ma...,movie,The Gold Rush,The Gold Rush,0.0,1925.0,NaN,95,"Adventure,Comedy,Drama"
2,tt0017136,Brigitte Helm|Alfred Abel|Gustav Fröhlich|Rudo...,movie,Metropolis,Metropolis,0.0,1927.0,NaN,153,"Drama,Sci-Fi"
3,tt0021749,Charles Chaplin|Virginia Cherrill|Florence Lee...,movie,City Lights,City Lights,0.0,1931.0,NaN,87,"Comedy,Drama,Romance"
4,tt0027977,Al Ernest Garcia|Charles Chaplin|Paulette Godd...,movie,Modern Times,Modern Times,0.0,1936.0,NaN,87,"Comedy,Drama,Romance"
...,...,...,...,...,...,...,...,...,...,...
1992,tt9639470,Anya Taylor-Joy|Thomasin McKenzie|Matt Smith|D...,movie,Last Night in Soho,Last Night in Soho,0.0,2021.0,NaN,116,"Drama,Horror,Mystery"
1993,tt9764362,Ralph Fiennes|Nicholas Hoult|Anya Taylor-Joy|H...,movie,The Menu,The Menu,0.0,2022.0,NaN,107,"Horror,Thriller"
1994,tt9770150,Frances McDormand|David Strathairn|Linda May|G...,movie,Nomadland,Nomadland,0.0,2020.0,NaN,107,Drama
1995,tt9777666,Chris Pratt|J.K. Simmons|Yvonne Strahovski|Bet...,movie,The Tomorrow War,The Tomorrow War,0.0,2021.0,NaN,138,"Action,Adventure,Drama"


In [5]:
from rdflib import Graph
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://localhost:7200/repositories/wikidata")

def gen():
    QUERY = '''
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT DISTINCT
    ?title ?tconst
    (GROUP_CONCAT(DISTINCT ?director; SEPARATOR="|") as ?directors)
    (GROUP_CONCAT(DISTINCT ?composer; SEPARATOR="|") as ?composers)
    (GROUP_CONCAT(DISTINCT ?genre; SEPARATOR="|") as ?genres)
    (GROUP_CONCAT(DISTINCT ?subject; SEPARATOR="|") as ?subjects)
    (GROUP_CONCAT(DISTINCT ?period; SEPARATOR="|") as ?periods)
    (GROUP_CONCAT(DISTINCT ?location; SEPARATOR="|") as ?locations)

    WHERE {
        ?film wdt:P31 wd:Q11424 .
        ?film rdfs:label ?title .
        ?film wdt:P345 ?tconst .
        OPTIONAL { ?film wdt:P57 ?d . ?d rdfs:label ?director . }
        OPTIONAL { ?film wdt:P86 ?c . ?c rdfs:label ?composer . }
        OPTIONAL { ?film wdt:P136 ?g . ?g rdfs:label ?genre . }
        OPTIONAL { ?film wdt:P921 ?s . ?s rdfs:label ?subject . }
        OPTIONAL { ?film wdt:P180 ?s2 . ?s2 rdfs:label ?subject . }
        OPTIONAL { ?film wdt:P2401 ?p . ?p rdfs:label ?period . }
        OPTIONAL { ?film wdt:P840 ?l . ?l rdfs:label ?location . }
    }
    GROUP BY ?title ?tconst
    '''

    sparql.setQuery(QUERY)
    sparql.setReturnFormat(JSON)
    ret = sparql.queryAndConvert()

    for row in ret['results']['bindings']:
        yield {k: v['value'] for k, v in row.items()}


df2 = pd.DataFrame.from_records(gen())
df2

,title,tconst,directors,composers,genres,subjects,periods,locations
0,The Intouchables,tt1675434,Éric Toledano|Olivier Nakache,Ludovico Einaudi,film based on literature|biographical film|com...,,,Paris
1,Copernicus,tt0070278,Czesław Petelski|Ewa Petelska,Jerzy Maksymiuk,historical film,,,
2,12 Angry Men,tt0050083,Sidney Lumet,Kenyon Hopkins,drama film|Huis clos|trial film,jury trial|distinction|capital punishment,,Manhattan
3,Nalai Manithan,tt4168940,Velu Prabhakaran,,romance film|horror film|action film,,,Chennai
4,Labyrinth,tt0091369,Jim Henson,Trevor Jones,fantasy film|coming-of-age fiction|adventure f...,,,
...,...,...,...,...,...,...,...,...
39111,Carbon,tt6183834,Olivier Marchal,,crime film|thriller film,,,
39112,Green,tt3223162,,,documentary film,,,
39113,Knock,tt6515342,Lorraine Lévy,Cyrille Aufort,comedy film,,,
39114,Going the Limit,tt0133811,Chester Withey,,silent film,,,


In [6]:
df3 = top.merge(df2, on='tconst')
df3

,tconst,primaryName,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_x,title,directors,composers,genres_y,subjects,periods,locations
0,tt0015864,Charles Chaplin|Henry Bergman|Frank Aderias|Ma...,movie,The Gold Rush,The Gold Rush,0.0,1925.0,NaN,95,"Adventure,Comedy,Drama",The Gold Rush,Charlie Chaplin,Charlie Chaplin,comedy film|silent film|romantic comedy|social...,,,
1,tt0032553,Charles Chaplin|Paulette Goddard|Grace Hayle|J...,movie,The Great Dictator,The Great Dictator,0.0,1940.0,NaN,125,"Comedy,Drama,War",The Great Dictator,Charlie Chaplin,Charlie Chaplin|Meredith Willson,drama film|comedy drama|war film|anti-war film...,antisemitism|fascism,,Osterlich|Bacteria|Tomainia
2,tt0032976,Laurence Olivier|Joan Fontaine|George Sanders|...,movie,Rebecca,Rebecca,0.0,1940.0,NaN,130,"Drama,Film-Noir,Mystery",Rebecca,Alfred Hitchcock,Franz Waxman,drama film|romance film|film based on a novel|...,,,Cornwall
3,tt0041959,Orson Welles|Joseph Cotten|Alida Valli|Trevor ...,movie,The Third Man,The Third Man,0.0,1949.0,NaN,104,"Film-Noir,Mystery,Thriller",The Third Man,Carol Reed,Anton Karas,drama film|adventure film|thriller film|myster...,foreigner|hoax|profiteering|reconstruction,,Vienna|Allied-occupied Austria
4,tt0042192,George Sanders|Bette Davis|Anne Baxter|Celeste...,movie,All About Eve,All About Eve,0.0,1950.0,NaN,138,Drama,All About Eve,Joseph L. Mankiewicz,Alfred Newman,film based on literature|comedy film|drama film,,,New York City|Connecticut
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,tt7349950,Jessica Chastain|Bill Hader|James McAvoy|Isaia...,movie,It Chapter Two,It Chapter Two,0.0,2019.0,NaN,169,"Drama,Fantasy,Horror",It: Chapter Two,Andrés Muschietti,Benjamin Wallfisch,horror film|film based on a novel|LGBT-related...,,,Maine
445,tt7556122,Charlize Theron|Marwan Kenzari|Matthias Schoen...,movie,The Old Guard,The Old Guard,0.0,2020.0,NaN,125,"Action,Thriller",The Old Guard,Gina Prince-Bythewood,Hauschka|Dustin O'Halloran,action film|speculative/fantastic fiction film...,immortality,,London|Afghanistan|South Sudan|Marrakesh|Gouss...
446,tt8772262,Florence Pugh|Jack Reynor|Vilhelm Blomgren|Wil...,movie,Midsommar,Midsommar,0.0,2019.0,NaN,148,"Drama,Horror,Mystery",Midsommar,Ari Aster,,drama film|horror film|thriller film,,,Sweden|Hårga
447,tt9114286,Lupita Nyong'o|Danai Gurira|Winston Duke|Letit...,movie,Black Panther: Wakanda Forever,Black Panther: Wakanda Forever,0.0,2022.0,NaN,161,"Action,Adventure,Drama",Black Panther: Wakanda Forever,Ryan Coogler,Ludwig Göransson,action film,,,Culiacán


In [7]:
import json 

film_data = {}

for row in df3.itertuples():
    title = row.title
    year = int(row.startYear)
    display = f'{title} ({year})'
    film = {
        'title': title,
        'imdb': row.tconst,
        'decade': str(int(year / 10) * 10) + 's',
        'actors': sorted(set(
            row.primaryName.split('|')
        )),
    }

    film['directors'] = sorted(set(row.directors.split('|')), key=str.casefold) or None
    film['composers'] = sorted(filter(bool, set(row.composers.split('|'))), key=str.casefold) or None
    film['settings'] = sorted(filter(bool, set(row.periods.split('|')) | set(row.locations.split('|'))),
        key=str.casefold
    ) or None
    film['genres'] = sorted(
        set(g.replace(' film', '').lower() for g in row.genres_y.split('|'))
        |
        set(g.lower() for g in row.genres_x.split(',')),
        key=str.casefold
    ) or None
    film['subjects'] = sorted(filter(bool, set(row.subjects.split('|'))), key=str.casefold) or None

    if len(film) > 2:
        film_data[display] = film

with open('films.json', 'w') as f:
    json.dump(film_data, f, indent=2)